## This notebook has the code to append respective frequency domain features (FFT, avgPow, totalPow, weighted-avg-frequencies) to the corresponding time-series segments.

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical
import tensorflow as tf
from scipy.signal import butter, filtfilt

# Assuming you have your data X and labels y defined
# Make sure that your data is appropriately shaped, e.g., (num_samples, sequence_length, num_features)
freq8 = pd.read_csv('/content/drive/MyDrive/LeBrain - Neureality 2024/ssvep/sunday data/latest/ssvep_8hz.csv')
freq15 = pd.read_csv('/content/drive/MyDrive/LeBrain - Neureality 2024/ssvep/sunday data/latest/ssvep_15hz.csv')
freq25 = pd.read_csv('/content/drive/MyDrive/LeBrain - Neureality 2024/ssvep/sunday data/latest/ssvep_25hz.csv')
freq35 = pd.read_csv('/content/drive/MyDrive/LeBrain - Neureality 2024/ssvep/sunday data/latest/ssvep_35hz.csv')

freq8 = freq8.iloc[1200:, 4:8]
freq15 = freq15.iloc[1200:, 4:8]
freq25 = freq25.iloc[1200:, 4:8]
freq35 = freq35.iloc[1200:, 4:8]

freq8.columns = ['0', '1', '2', '3']
freq15.columns = ['0', '1', '2', '3']
freq25.columns = ['0', '1', '2', '3']
freq35.columns = ['0', '1', '2', '3']

grouped_freq8 = freq8
grouped_freq15 = freq15
grouped_freq25 = freq25
grouped_freq35 = freq35

grouped_freq8['label'] = 1
grouped_freq15['label'] = 2
grouped_freq25['label'] = 3
grouped_freq35['label'] = 4

grouped_freq8=grouped_freq8[:18125]
grouped_freq15=grouped_freq15[:21250]
grouped_freq25=grouped_freq25[:21250]
grouped_freq35=grouped_freq35[:21250]

stacked_df = pd.concat([grouped_freq8, grouped_freq15, grouped_freq25, grouped_freq35], axis=0, ignore_index=True)

eeg_channels = stacked_df.iloc[:, 0:4].copy()

lowcut = 4.0  # Lower cutoff frequency in Hz
highcut = 40.0  # Upper cutoff frequency in Hz
sampling_rate = 1000  # Adjust this to your actual sampling rate
nyquist = 0.5 * sampling_rate
low = lowcut / nyquist
high = highcut / nyquist
b, a = butter(4, [low, high], btype='band')

# Apply the bandpass filter to each column
filtered_channels = eeg_channels.apply(lambda column: filtfilt(b, a, column))

# Combine the filtered channels with the labels from the original DataFrame
filtered_stacked_df = pd.concat([filtered_channels, stacked_df['label']], axis=1)

# Display the resulting DataFrame
print(filtered_stacked_df)

sequence_length = 625
num_features = 4

appended_arrays = []

df_attributes = filtered_stacked_df.drop(columns=['label'])

for i in range(0, len(df_attributes) - sequence_length + 1, sequence_length):
    sequence_data = df_attributes.iloc[i:i + sequence_length].values

    # Reshape the sequence_data to match the input shape for Conv1D
    # Calculate real FFT of the sequence
    sequence_fft = np.abs(np.fft.fft(sequence_data))

    # Calculate average power, total power, and average frequency
    average_power = np.mean(sequence_fft**2, axis = 0).reshape(1, -1)
    total_power = np.sum(sequence_fft**2, axis = 0).reshape(1, -1)
    frequencies = np.fft.fftfreq(len(sequence_data[0]))

    # Weighted average frequency using dot product along the correct axis
    weighted_avg_frequency = np.sum(frequencies * sequence_fft**2, axis=0).reshape(1, -1) / np.sum(sequence_fft**2, axis=0).reshape(1, -1)


    # print(sequence_fft.shape)
    # print(sequence_data.shape)
    # print(average_power.shape)
    # print(total_power.shape)
    # print(weighted_avg_frequency.shape)

    # Append sequence, FFT, average power, total power, and weighted average frequency to X
    new_array = np.concatenate([sequence_data, sequence_fft, average_power, total_power, weighted_avg_frequency], axis=0)
    # print(new_array.shape)

    appended_arrays.append(new_array)

X = np.array(appended_arrays)

print(X.shape)

X = X.reshape((X.shape[0], X.shape[1], X.shape[2]))

print(X.shape)

# Convert the list to a NumPy array
#y = np.array(y)
# Split data into training and testing sets

y = [1] * 29 + [2] * 34 + [3] * 34 + [4] * 34
y = np.array(y)
one_hot_y = tf.one_hot(y, depth=4).numpy()

X_train, X_test, y_train, y_test = train_test_split(X, one_hot_y, test_size=0.3, random_state=42)

# # Create and apply Conv1D layers
# model = Sequential([
#     Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=(sequence_length*2+3, num_features, 1)),
#     MaxPooling1D(pool_size=2),
#     Conv1D(filters=64, kernel_size=3, activation='relu'),
#     MaxPooling1D(pool_size=2),
#     Flatten(),
#     Dense(128, activation='relu'),
#     Dropout(0.5),
#     Dense(4, activation='softmax')
# ])

# model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


# # Train the model
# # model.fit(sequence_data, y, epochs=30, batch_size=2, validation_split=0.2)
# # Train the model
# model.fit(X_train, y_train, epochs=30, batch_size=2, validation_data=(X_test, y_test))

# # Evaluate the model
# loss, accuracy = model.evaluate(X_test, y_test)
# print(f'Test Loss: {loss}, Test Accuracy: {accuracy}')


              0         1         2         3  label
0      0.586344  0.305874  1.148232  0.527211      1
1      1.461505  0.873929  1.930340  1.098227      1
2      2.320186  1.439350  2.699922  1.657165      1
3      3.143926  1.991526  3.440917  2.192087      1
4      3.915966  2.520977  4.139307  2.692513      1
...         ...       ...       ...       ...    ...
81870  7.324081  7.103811  7.794086  5.006403      4
81871  6.493461  6.139875  6.619907  4.444977      4
81872  5.453382  4.977609  5.252977  3.746530      4
81873  4.259528  3.674369  3.755854  2.947235      4
81874  2.973907  2.293866  2.196679  2.087799      4

[81875 rows x 5 columns]
(131, 1253, 4)
(131, 1253, 4)


In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv1D, LSTM, Dense, Dropout, concatenate, GlobalAveragePooling1D
from tensorflow.keras.regularizers import l2

input_layer = Input(shape=(sequence_length*2+3, num_features))

# 1st Conv1D layer with MaxPooling and Dropout
conv1d_1 = Conv1D(filters=64, kernel_size=6, activation='relu', kernel_regularizer=l2(0.1))(input_layer)
maxpooling_1 = MaxPooling1D(pool_size=2)(conv1d_1)
dropout_1 = Dropout(0.2)(maxpooling_1)

# 2nd Conv1D layer with MaxPooling and Dropout
conv1d_2 = Conv1D(filters=32, kernel_size=6, activation='relu', kernel_regularizer=l2(0.1))(dropout_1)
maxpooling_2 = MaxPooling1D(pool_size=2)(conv1d_2)
dropout_2 = Dropout(0.2)(maxpooling_2)

# # 3rd Conv1D layer with MaxPooling and Dropout
# conv1d_3 = Conv1D(filters=256, kernel_size=6, activation='relu', kernel_regularizer=l2(0.01))(dropout_2)
# maxpooling_3 = MaxPooling1D(pool_size=2)(conv1d_3)
# dropout_3 = Dropout(0.2)(maxpooling_3)

# Flatten layer
flatten_layer = Flatten()(dropout_2)

# 1st Dense layer
dense_1 = Dense(16, activation='relu', kernel_regularizer=l2(0.01))(flatten_layer)
dropout_dense_1 = Dropout(0.2)(dense_1)

# # 2nd Dense layer
# dense_2 = Dense(8, activation='relu', kernel_regularizer=l2(0.01))(dropout_dense_1)
# dropout_dense_2 = Dropout(0.2)(dense_2)

# Output Dense layer
output_layer = Dense(4, activation='softmax')(dropout_dense_1)

# Create the model
model = Model(inputs=input_layer, outputs=output_layer)

# Compile the model
model.compile(optimizer='adam', loss='kullback_leibler_divergence', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=100, batch_size=4, validation_data=(X_test, y_test))

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Loss: {np.mean(loss)}, Test Accuracy: {np.mean(accuracy)}')

Epoch 1/100
23/23 [==============================] - 5s 16ms/step - loss: 7.2450 - accuracy: 0.2198 - val_loss: 5.8483 - val_accuracy: 0.2250
Epoch 2/100
23/23 [==============================] - 0s 6ms/step - loss: 5.2665 - accuracy: 0.3516 - val_loss: 5.1895 - val_accuracy: 0.2250
Epoch 3/100
23/23 [==============================] - 0s 6ms/step - loss: 4.5580 - accuracy: 0.3297 - val_loss: 4.5160 - val_accuracy: 0.3250
Epoch 4/100
23/23 [==============================] - 0s 6ms/step - loss: 3.9120 - accuracy: 0.4396 - val_loss: 4.0941 - val_accuracy: 0.3250
Epoch 5/100
23/23 [==============================] - 0s 6ms/step - loss: 3.3375 - accuracy: 0.5275 - val_loss: 3.6163 - val_accuracy: 0.3250
Epoch 6/100
23/23 [==============================] - 0s 6ms/step - loss: 2.9398 - accuracy: 0.5055 - val_loss: 3.2609 - val_accuracy: 0.3750
Epoch 7/100
23/23 [==============================] - 0s 6ms/step - loss: 2.4914 - accuracy: 0.6044 - val_loss: 3.1571 - val_accuracy: 0.2000
Epoch 8/100
